In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Import data from csv file

In [ ]:
data_train = pd.read_csv('/kaggle/input/loan-default-prediction/train_v2.csv.zip')

In [ ]:
#checking the first 5 rows of training data
data_train.head()

In [ ]:
#shape of training data
data_train.shape

In [ ]:
#information about data and it's types
data_train.info()

There are 653 of float type, 99 of integral type and 19 are categorical type

In [ ]:
#selecting categorical columns
data_cat = data_train.select_dtypes(include=['object']).copy()

In [ ]:
data_cat.head(10)

In [ ]:
print(data_cat.columns)
print(data_cat['f137'].value_counts())

if we observe above columns, there are no f776 or f777 features but as per data, they are categorical too and the values of categorical columns above are way too high and encoding them will be difficult as they are in large counts while being unique for different consumers. Hence, we will deal only with numerical data.

In [ ]:
#extracting numerical columns from training data
data_num = data_train.select_dtypes(include=['float64', 'int64']).copy()

In [ ]:
data_num.head()

In [ ]:
data_num.columns

In [ ]:
#checking for any missing values in each column
data_num.isnull().sum()

In [ ]:
#imputing missing values with mean of that column
data_num_imputed = data_num.fillna(data_num.mean(), inplace=False)

In [ ]:
#checking for missing values
data_num_imputed.isnull().sum()

As we can see that, all missing values have been imputed.

In [ ]:
#removing id column from imputed data
data_num_imputed.drop(columns='id', inplace=True)

In [ ]:
data_num_imputed.head()

The data is arranged from old to new in time dimension. Hence, we will shuffle it to make a randomly sampled data.

In [ ]:
#shuffling dataframe in random order to remove any bias due to time sampling
data_num_shuffled = data_num_imputed.sample(frac=1, random_state=2)
data_num_shuffled.head()

In [ ]:
#seperating loss column and calculating correlation matrix for features
loss = data_num_shuffled['loss'].copy()
data_num_shuffled.drop(columns='loss', inplace=True)
corr_matrix = data_num_shuffled.corr().abs()
corr_matrix.head()
corr_matrix.shape

In [ ]:
#taking upper triangular part of correlation matrix 
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head()

In [ ]:
# extracting highly correlated features with correlation coefficient of more than 0.8
threshold = 0.8
col_to_drop = [column for column in upper.columns if any(upper[column]>threshold)]
len(col_to_drop)

In [ ]:
# dropping highly correlated columns
data_num_shuffled.drop(columns=col_to_drop, inplace=True)
data_num_shuffled.columns

In [ ]:
# finding correlation of features with target values of loss and convert into a dataframe
corr_tar = data_num_shuffled.corrwith(loss).sort_values()
print(corr_tar.head(30))
print(corr_tar.tail(30))
corr_tar_df = corr_tar.to_frame().transpose()
corr_tar_df.isna()

In [ ]:
# extracting features having NaN value correlation with loss to remove them 
col_to_drop_1 = corr_tar_df.columns[corr_tar_df.isna().any()].to_list()
print(len(col_to_drop_1))
print(col_to_drop_1)

In [ ]:
data_num_shuffled.drop(columns=col_to_drop_1, inplace=True)

In [ ]:
data_num_shuffled.columns

In [ ]:
# creating a copy of final features dataframe 
X_feat = data_num_shuffled.copy()

In [ ]:
#creating a copy of loss dataframe 
Y_tar = loss.copy()

In [ ]:
# making target or loss a binary valued variable for better classification and interpretation
Y_tar[Y_tar>0] = 1
Y_tar.value_counts()

In [ ]:
# splitting data into train and test splits and normalizing data for better training
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_feat, Y_tar, test_size=0.2, random_state=42)
ss = StandardScaler()
X_train = ss.fit_transform(x_train)
X_test = ss.transform(x_test)

In [ ]:
# converting y to one-dimensional array
y_train = np.array(y_train).reshape((-1, ))
y_test = np.array(y_test).reshape((-1, ))

In [ ]:
# function for calculating cross validation scores for evaluating model performance on train data and test data
def scores(X_train, y_train, X_test, y_test, model):
    from sklearn.model_selection import cross_val_score, ShuffleSplit
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42) # shufflesplit is a cross validation strategy that is best used when n_features < n_samples
    train_scores = cross_val_score(estimator = model, X = X_train, y = y_train, cv = cv)
    test_scores = cross_val_score(estimator = model, X = X_test, y = y_test, cv = cv)
    return [round(train_scores.mean(), 4), round(test_scores.mean(),4)]

In [ ]:
# function to fit and evaluate model performance
def fit_and_evaluate(model):
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions and evalute
    y_pred = model.predict(X_test)
    cross_val_scores = scores(X_train, y_train, X_test, y_test, model)
    
    # calculate accuracy and f_score for model performance
    from sklearn.metrics import accuracy_score, f1_score
    accuracy = accuracy_score(y_test, y_pred, normalize=True)
    f_score = f1_score(y_test, y_pred, average='macro')
    
    # dictionary for storing all the above values
    metric = dict()
    metric['cv_scores'] = cross_val_scores
    metric['accuracy'] = round(accuracy,4)
    metric['f_score'] = round(f_score,4)
    
    # Return the performance metric 
    return metric

Here, we will compare three classification algorithms for their performance and choose the best one. We considered logistic regression, random forest and xgboost classifiers to compare T 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# logistic regression 
logistic = LogisticRegression(max_iter = 50, random_state=42)
log_c = fit_and_evaluate(logistic)
print('performance of logistic regression on train and test data:', log_c['cv_scores'])
log_c.pop('cv_scores')
print(log_c)
df_log = pd.DataFrame(log_c, index=[0])
print(df_log)

# Random forest classification
RFC = RandomForestClassifier(n_estimators = 15, random_state=42)
random = fit_and_evaluate(RFC)
print('performance of Random Forest Classifier on train and test data:', random['cv_scores'])
random.pop('cv_scores')
print(random)
df_rand = pd.DataFrame(random, index=[1])
print(df_rand)

# Xgboost classification
gb = XGBClassifier()
gb_c = fit_and_evaluate(gb)
print('performance of XGB Classifier on train and test data:', gb_c['cv_scores'])
gb_c.pop('cv_scores')
print(gb_c)
df_gb = pd.DataFrame(gb_c, index=[2])
print(df_gb)

frames = [df_log, df_rand, df_gb]
model_compare_df = pd.concat(frames)

print(model_compare_df)

Cross validation scores for all three classifiers are almost same for both training data and test data which implies that our models are good and don't overfit. We will proceed with random forest classifier as it has best F score and also, it is better indicator than accuracy for model performance as we know it.

In [ ]:
# finding least important features from random forest classification so as to see if we can improve our model performance or we can get same performance with less features
feature_importances = RFC.feature_importances_
feature_importances = pd.DataFrame({'feature': list(X_feat.columns), 'importance': feature_importances}).sort_values('importance', ascending = True)
print(feature_importances['importance'].mean()/2) # threshold for filtering least important features
least_important_feat = list(feature_importances[feature_importances['importance'] < 0.0025]['feature'])
len(least_important_feat)

In [ ]:
# creating dataframe with important features
X_feat_imp = X_feat.drop(columns=least_important_feat, inplace=False)
len(X_feat_imp.columns)

In [ ]:
# splitting data into train and test
x_train_imp, x_test_imp, y_train, y_test = train_test_split(X_feat_imp, Y_tar, test_size=0.2, random_state=42)
ss = StandardScaler()
X_train_imp = ss.fit_transform(x_train_imp)
X_test_imp = ss.transform(x_test_imp)

In [ ]:
# converting y to one-dimensional array
y_train = np.array(y_train).reshape((-1, ))
y_test = np.array(y_test).reshape((-1, ))

In [ ]:
# function to fit and evaluate model performance with important features
def fit_and_evaluate_imp(model):
    
    # Train the model
    model.fit(X_train_imp, y_train)
    
    # Make predictions and evalute
    y_pred = model.predict(X_test_imp)
    cross_val_scores = scores(X_train_imp, y_train, X_test_imp, y_test, model)
    
    # calculate accuracy and f_score for model performance
    from sklearn.metrics import accuracy_score, f1_score
    accuracy = accuracy_score(y_test, y_pred, normalize=True)
    f_score = f1_score(y_test, y_pred, average='macro')
    
    # dictionary for storing all the above values
    metric = dict()
    metric['cv_scores'] = cross_val_scores
    metric['accuracy'] = round(accuracy,4)
    metric['f_score'] = round(f_score,4)
    
    # Return the performance metric 
    return metric

In [ ]:
# Random forest classification with important features
RFC_imp = RandomForestClassifier(n_estimators = 15, random_state=42)
random_imp = fit_and_evaluate_imp(RFC_imp)
print('performance of Random Forest Classifier on train and test data:', random_imp['cv_scores'])
random_imp.pop('cv_scores')
print(random_imp)
df_rand_imp = pd.DataFrame(random_imp, index=[0])
print(df_rand_imp)

F score with most important features has improved. Now, we will preprocess test data for prediction

In [ ]:
data_test = pd.read_csv('/kaggle/input/loan-default-prediction/test_v2.csv.zip')

In [ ]:
data_test.shape

In [ ]:
data_test.columns

In [ ]:
print(X_feat.columns)
print(X_feat_imp.columns)

In [ ]:
# getting columns that are used for training
test_feat = data_test[X_feat.columns]
test_feat_imp = data_test[X_feat_imp.columns]
print(test_feat.columns)
print(test_feat_imp.columns)

In [ ]:
print(test_feat.isnull().sum())
print(test_feat_imp.isnull().sum())

In [ ]:
test_feat_imputed = test_feat.fillna(test_feat.mean(),inplace=False)
test_feat_imp_imputed = test_feat_imp.fillna(test_feat_imp.mean(),inplace=False)

In [ ]:
print(test_feat_imputed.isnull().sum())
print(test_feat_imp_imputed.isnull().sum())

In [ ]:
ss = StandardScaler()
test_feat_scaled = ss.fit_transform(test_feat_imputed)
test_feat_imp_scaled = ss.fit_transform(test_feat_imp_imputed)

In [ ]:
predicted_val = RFC.predict(test_feat_scaled)
predicted_val_imp = RFC_imp.predict(test_feat_imp_scaled)
df_val = pd.DataFrame({'loss_as_binary':predicted_val})
df_val_imp = pd.DataFrame({'loss_as_binary':predicted_val_imp})
print(df_val)
print(df_val_imp)

In [ ]:
predicted_prob = RFC.predict_proba(test_feat_scaled)
predicted_prob_imp = RFC_imp.predict_proba(test_feat_imp_scaled)
df_prob = pd.DataFrame({'no_default': predicted_prob[:, 0], 'default': predicted_prob[:, 1]})
df_prob_imp = pd.DataFrame({'no_default': predicted_prob_imp[:, 0], 'default': predicted_prob_imp[:, 1]})
print(df_prob)
print(df_prob_imp)

In [ ]:
submission = pd.DataFrame()
submission['id'] = data_test.id
submission['loss_as_binary'] = df_val.loss_as_binary
submission['default_prob'] = round(df_prob.default,2)
print(submission)

In [ ]:
submission.head()

In [ ]:
Submission = submission.to_csv(index=False)

In [ ]:
submission_imp = pd.DataFrame()
submission_imp['id'] = data_test.id
submission_imp['loss_as_binary'] = df_val_imp.loss_as_binary
submission_imp['default_prob'] = round(df_prob_imp.default,2)
print(submission_imp)

In [ ]:
submission_imp.head()

In [ ]:
Submission_imp = submission_imp.to_csv(index=False)

In [ ]:
import os 
os.chdir(r'/kaggle/working')

In [ ]:
submission.to_csv(r'SUBMISSION.csv',index=False)
submission_imp.to_csv(r'SUBMISSION_imp.csv',index=False)

In [ ]:
from IPython.display import FileLink
FileLink(r'SUBMISSION.csv')

In [ ]:
FileLink(r'SUBMISSION_imp.csv')